# Recurrent Neural Network
***
**Name**: Timothy Mason
***

## Goal
The goal of this assignment is to use TensorFlow to build some recurrent neural nets (RNNs) and to understand their limitations through experimentation.

## The Task
You will implement a recurrent neural net to learn the parity operator. The net will have a single input unit and a single output unit, and a fully-connected layer of H hidden units. The inputs and target outputs are binary. When an input sequence is presented, the output state at the end of the sequence should be a parity bit: output should be 1 if the input has an odd number of '1' values. For example, the sequence 1-0-0-1-0-1 should yield output 1 and the sequence 0-0-0-0-1-1 should yield output 0. Note that a target is given only at the end of each sequence. (Parity is easy to learn if there is a target at each step that indicates parity given the sequence so far.)

Parity is a hard problem for neural nets to learn because very similar inputs produce different outputs, and very dissimilar inputs can produce the same output.

The aspects of the task we will manipulate are:  H, the number of hidden units, N, the length of the input strings, and the activation function for the hidden units, either tanh or LSTM-style neurons. The output neuron should have a logistic activation function.

Tip: get started early.  Depending on your system, these nets can take several minutes to train.  Exploring hyperparameters (such as training rate) will be critical for success.


### Some Help
Below are some helper codes to:
- generate input strings and their parity.
- provide a callback to trigger early stopping during training
- plotting

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np
from matplotlib import pyplot

def gen_binary_sequences(num, length):
  '''
  Generate :num: sequences of length :length:.
  '''
  return(np.random.randint(0, 2, size=(num, length), dtype=np.int32))

def calc_parity(seqs):
  '''
  Calculate sequence parity (1 if odd number of 1s, 0 if even number of 1s)
  '''
  return(seqs.sum(axis=-1) % 2)


class create_accuracy_callback(tf.keras.callbacks.Callback):
    '''
    Callback function to stop training at 100% accuracy
    '''
    def on_epoch_end(self, epoch, logs={}):
#         print(f"epoch {epoch}  logs: {logs}")
#         breakpoint()
        if(logs.get('accuracy') == 1.0 and logs.get('val_accuracy') == 1.0):
            self.model.stop_training = True


def plot_accuracies(accuracies,Ns,Hs):
  '''
  Make a graph of mean % correct (and standard error) on the test set for the different values of H and N.
  Input: ndarray of |Hs|x|Ns|x(reps)
         actual values of Ns and Hs
  '''
  lenH,lenN,lenreps = accuracies.shape
  assert(lenH == len(Hs))
  assert(lenN == len(Ns))
  accuracies_mean = accuracies.mean(axis=2)
  accuracies_std = accuracies.std(axis=2)
  accuracies_stderr = accuracies_std/np.sqrt(lenreps)
  # plot
  fig = pyplot.figure()
  ax = fig.add_subplot(111)
  ax.axhline(0.5,linestyle="--",color="gray") # chance baseline
  centers = np.arange(lenN)
  for Hindex, H in enumerate(Hs):
    ax.bar(centers + 0.8/lenH*(Hindex-(lenH-1)/2),
           accuracies_mean[Hindex], 0.8/lenH, yerr=accuracies_stderr[Hindex],
           alpha=0.5, label=f"{H} hidden units")
  ax.set_xlabel("sequence length")
  pyplot.xticks(centers,Ns)
  ax.set_ylabel("accuracy")
  pyplot.legend(loc="lower left")

TensorFlow 2.x selected.


## Part 1
**Part 1.a**<br>
Fill in the code to create a net given H and N using the tanh activation function for the hidden units.  Keras has a number of RNN helper functions, although you can also write your own custom layers.

Remember that the net should only take one bit of input at a time from the input sequence, and output one logistic value (between 0 and 1) only after the input sequence is complete.

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras import layers

def build_model(N,H):
    '''
    Builder for an RNN model. Model inputs are binary sequences of length :N:.

    At each sequence position, the input and prior state should be fully connected 
    to :H: hidden units with tanh activation.

    The output of the last state of the RNN should be fully connected to a single
    unit with logistic activation, to perform the final classification of the 
    sequence.
    '''

    # based on example Keras RNN code from https://www.tensorflow.org/guide/keras/rnn

    model = tf.keras.Sequential()
    model.add(layers.Embedding(input_dim=N, output_dim=H))
    model.add(layers.SimpleRNN(H))   # defaults are good (tanh activation, use a bias vector)
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

**Part 1.b**<br>
Then fill in the code to train several such nets.  Each repetition should randomize the initial weights and generate a random training set of 10000 examples of length N as well as a random test set of 10000 examples of length N.  Save 10% of the training as validation, and use at least the provided check_accuracy callback as an early stopping condition.

Train nets for H ∈ {5, 25} and for N ∈ {5, 10, 15, 20}.  For each combination of H and N, run 10 replications of your simulation.  You will also need to try to find helpful learning rates; don't be surprised if your training is prone to cycles of stagnation for hundreds of epochs before quickly learning.

In [0]:
BATCH = 1000
MAX_EPOCHS = 2000

check_accuracy = create_accuracy_callback()
def train_model(model, x_train, y_train, x_test, y_test):
    '''
    Compiles and trains the :model:.
    '''
    # start by compiling the model; accuracy should be a tracked metric
    model.compile(loss='binary_crossentropy', metrics=['accuracy'])

    # train model
    model.fit( x_train,
              y_train, 
              batch_size=BATCH, 
              epochs=MAX_EPOCHS, 
              verbose=1, 
              validation_data=(x_test, y_test),
              callbacks=[check_accuracy] )

In [4]:
Hs = [5,25]
Ns = [5] # switch to [5,10,20,40] when things are working
REPS = 10
SAMPLES = 10000
# breakpoint()

test_accs = np.zeros([len(Hs),len(Ns),REPS])
for Nindex, N in enumerate(Ns):
    for rep in range(REPS):
        # (data is reusable across changes to the number of hidden units)
        # generate a random training set of 10000 examples of length N
        x_train = gen_binary_sequences(SAMPLES, N)
        y_train = calc_parity(x_train)

        # generate a random test set of 10000 examples of length N
        x_test = gen_binary_sequences(SAMPLES, N)
        y_test = calc_parity(x_train)

        for Hindex, H in enumerate(Hs):
            print(f"starting N={N}, H={H}, rep {rep}/{REPS}")
            # build model
            model = build_model(N,H)
            # train model
            train_model(model, x_train, y_train, x_test, y_test)
            # test model (extracting test accuracy as test_acc)
            test_acc = model.evaluate( x=x_test, 
                      y=y_test, 
                      batch_size=BATCH, 
                      verbose=1, 
                      callbacks=[check_accuracy] )[1]

            test_accs[Hindex,Nindex,rep] = test_acc

starting N=5, H=5, rep 0/10
Train on 10000 samples, validate on 10000 samples
Epoch 1/2000
10000/10000 [==============================] - 1s 123us/sample - loss: 0.6939 - accuracy: 0.5000 - val_loss: 0.6937 - val_accuracy: 0.4956
Epoch 2/2000
10000/10000 [==============================] - 0s 8us/sample - loss: 0.6935 - accuracy: 0.5121 - val_loss: 0.6935 - val_accuracy: 0.4914
Epoch 3/2000
10000/10000 [==============================] - 0s 8us/sample - loss: 0.6933 - accuracy: 0.5072 - val_loss: 0.6934 - val_accuracy: 0.4972
Epoch 4/2000
10000/10000 [==============================] - 0s 9us/sample - loss: 0.6932 - accuracy: 0.5008 - val_loss: 0.6933 - val_accuracy: 0.4972
Epoch 5/2000
10000/10000 [==============================] - 0s 10us/sample - loss: 0.6932 - accuracy: 0.5192 - val_loss: 0.6933 - val_accuracy: 0.4972
Epoch 6/2000
10000/10000 [==============================] - 0s 9us/sample - loss: 0.6932 - accuracy: 0.4848 - val_loss: 0.6933 - val_accuracy: 0.4930
Epoch 7/2000
10000/

NameError: ignored

**Part 1.c**<br>
Make the graph of mean % correct (and standard error) on the test set for the different values of H and N.

In [0]:
plot_accuracies(test_accs,Ns,Hs)

## Part 2
Repeat the experiments of Part 1, but use LSTM neurons instead of tanh neurons in the recurrent layer.  Comment on your experiences.